In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import torch

In [ ]:
mp_pose = mp.solutions.pose
poses = mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

In [ ]:
model = torch.hub.load("ultralytics/yolov5", "yolov5m", pretrained=True)

In [ ]:
video = cv2.VideoCapture(0)
flag = 0
frame_check = 10

In [ ]:
def angles(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180/np.pi)
    if angle > 180: angle = 360-angle
    return angle

In [ ]:
while True:
    _, image = video.read()

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)

    result = poses.process(image_rgb)

    h, w, c = image.shape
    if result.pose_landmarks:
        mp_draw.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        landmarks = result.pose_landmarks.landmark
        # print(landmarks)

        l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

        l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]

        l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
# ============================================================================================
        r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

        r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

        r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        # Calculate Angles
        l_ang = angles(l_shoulder, l_elbow, l_wrist)
        r_ang = angles(r_shoulder, r_elbow, r_wrist)

        if l_wrist[1]*h < l_elbow[1]*h < l_shoulder[1]*h and l_ang > 150:
            flag += 1
            if flag >= frame_check:
                cv2.putText(image, "Warning!!! Someone need Help", (20, 75), cv2.FONT_ITALIC, 1, (0, 255, 0))
                print(flag)
                print(l_ang)

        elif r_wrist[1]*h < r_elbow[1]*h < r_shoulder[1]*h and r_ang > 150:
            flag += 1
            if flag >= frame_check:
                cv2.putText(image, "Warning!!! Someone need Help", (20, 75), cv2.FONT_ITALIC, 1, (0, 255, 0))
                print(flag)
                print(r_ang)

        points = []
        for index, row in results.pandas().xyxy[0].iterrows():
            x1 = int(row["xmin"])
            y1 = int(row["ymin"])
            x2 = int(row["xmax"])
            y2 = int(row["ymax"])

            n = (row['name'])
            if "person" in n:
                if row["confidence"] > 0.25:
                    points.append([x1, y1, x2, y2])

                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
                    cv2.putText(image, str(row["confidence"]), (x1, y1),
                                cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))



    cv2.imshow("image", image)
    if cv2.waitKey(1) == ord("o"):break


video.release()
cv2.destroyAllWindows()
